<style>
    body, * {
        direction: rtl !important;
        text-align: right !important;
    }
</style>
# 📊 GroupBy וסידור בסדר יורד ב-PySpark DataFrame
[PySpark DataFrame groupBy and Sort by Descending Order](https://sparkbyexamples.com/pyspark/pyspark-dataframe-groupby-and-sort-by-descending-order/)
## 📚 מבוא

ב-DataFrame של PySpark, נראה כיצד לעשות את הפעולות הבאות ברצף:

1️⃣ **קיבוץ נתונים** - שימוש בפונקציית אגרגציה `sum()`

2️⃣ **סינון התוצאות** - שימוש ב-`filter()` על קבוצות

3️⃣ **מיון התוצאות** - שימוש ב-`sort()` או `orderBy()` לסידור בסדר עולה או יורד

---

## 🛠️ הכנת הסביבה והנתונים

בכדי להדגים את כל הפעולות הללו יחד, ניצור DataFrame של PySpark:

In [ ]:
# יבוא ספריות
from pyspark.sql import SparkSession

# יצירת Spark Session
spark = SparkSession.builder.appName("GroupByExample").getOrCreate()

# הגדרת נתוני דוגמה
simpleData = [
    ("James", "Sales", "NY", 90000, 34, 10000),
    ("Michael", "Sales", "NY", 86000, 56, 20000),
    ("Robert", "Sales", "CA", 81000, 30, 23000),
    ("Maria", "Finance", "CA", 90000, 24, 23000),
    ("Raman", "Finance", "CA", 99000, 40, 24000),
    ("Scott", "Finance", "NY", 83000, 36, 19000),
    ("Jen", "Finance", "NY", 79000, 53, 15000),
    ("Jeff", "Marketing", "CA", 80000, 25, 18000),
    ("Kumar", "Marketing", "NY", 91000, 50, 21000)
]

# הגדרת שמות העמודות
columns = ["employee_name", "department", "state", "salary", "age", "bonus"]

# יצירת DataFrame
df = spark.createDataFrame(data=simpleData, schema=columns)

# הצגת הנתונים
df.printSchema()
df.show(truncate=False)

---

## 1️⃣ שימוש ב-groupBy(), filter() ו-sort()

להלן דוגמה מלאה של איך לעשות קיבוץ נתונים ב-DataFrame, סינון וסידור בסדר יורד.

### 📋 הסבר על התהליך:

ראשית, נעשה **קיבוץ PySpark על DataFrame** באמצעות פונקציית אגרגציה `sum("salary")`.

`groupBy()` מחזיר אובייקט `GroupedData` שמכיל פונקציות אגרגציה כמו:
- `sum()`
- `max()`
- `min()`
- `mean()`
- `count()`

In [ ]:
# קיבוץ לפי state וחישוב סכום
from pyspark.sql.functions import sum, col, desc

df.groupBy("state").sum("salary").show()

### 🔍 הסבר מפורט על הדוגמה

בדוגמה למעלה, אי אפשר לתת שם alias לעמודת האגרגציה ושם העמודה לא ידידותי למשתמש.

מכיוון שזה מחזיר סכום של שכר, עדיף להשתמש בפונקציית SQL `sum()` של המחלקה הזו.

עכשיו נראה כיצד להשתמש ב-`agg()` כדי לתת alias באמצעות פונקציה `alias()` של המחלקה הזו.

כמו כן, כדי לסנן ולמיין לאחר הקיבוץ, צריך להכיר את שם העמודה המדויק, לכן נשתמש ב-`alias()` כדי לספק שם עמודה:

---

## 2️⃣ גישה משופרת - שימוש ב-alias()

להלן דוגמה לקיבוץ עם מתן שם ברור לעמודה:

In [ ]:
# Group by באמצעות agg() ו-alias
from pyspark.sql.functions import sum

dfGrouped = df.groupBy("state") \
    .agg(sum("salary").alias("sum_salary"))

dfGrouped.show()

**הסבר:**
- שימוש ב-`agg()` מאפשר לנו להשתמש בפונקציות אגרגציה מתקדמות
- שימוש ב-`alias()` נותן שם מתאים לעמודת התוצאה

---

## 3️⃣ סינון לאחר קיבוץ

להלן דוגמה לבחירת הנתונים שסכום השכר שלהם גדול מ-100,000:

In [ ]:
# סינון לאחר קיבוץ
dfFilter = dfGrouped.filter(col("sum_salary") > 100000)
dfFilter.show()

## 4️⃣ מיון בסדר עולה

עכשיו נשתמש ב-DataFrame sort() transformation של PySpark כדי למיין לפי סדר עולה בברירת מחדל:

In [ ]:
# מיון בסדר עולה

dfFilter.sort("sum_salary").show()

**הערה:** מיון בסדר עולה הוא ברירת המחדל.

---

## 5️⃣ מיון בסדר יורד - שימוש ב-desc()

בכדי לעשות סידור בסדר יורד, נשתמש בפונקציית Spark SQL `desc()`:

In [ ]:
# מיון בסדר יורד
from pyspark.sql.functions import desc

dfFilter.sort(desc("sum_salary")).show()

## 💡 חלופה: שימוש ב-SQL

לחלופין, אפשר גם להשתמש בביטוי SQL כדי להשיג את אותה התוצאה:

In [ ]:
# שימוש ב-SQL
df.createOrReplaceTempView("EMP")

spark.sql(
    "SELECT state, sum(salary) as sum_salary FROM EMP " +
    "GROUP BY state " +
    "HAVING sum(salary) > 100000 " +
    "ORDER BY sum_salary DESC"
).show()

**הערה:** גישת SQL מספקת תחביר מוכר למי שמכיר SQL מסורתי.

---

## 📝 קוד מלא - דוגמה מקיפה

להלן הקוד המלא המשלב את כל השלבים:

In [ ]:
# קוד מלא
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum, col, desc

spark = SparkSession.builder.appName("GroupByExample").getOrCreate()

simpleData = [
    ("James", "Sales", "NY", 90000, 34, 10000),
    ("Michael", "Sales", "NY", 86000, 56, 20000),
    ("Robert", "Sales", "CA", 81000, 30, 23000),
    ("Maria", "Finance", "CA", 90000, 24, 23000),
    ("Raman", "Finance", "CA", 99000, 40, 24000),
    ("Scott", "Finance", "NY", 83000, 36, 19000),
    ("Jen", "Finance", "NY", 79000, 53, 15000),
    ("Jeff", "Marketing", "CA", 80000, 25, 18000),
    ("Kumar", "Marketing", "NY", 91000, 50, 21000)
]

columns = ["employee_name", "department", "state", "salary", "age", "bonus"]
df = spark.createDataFrame(data=simpleData, schema=columns)
df.printSchema()
df.show(truncate=False)

# הצעה 1: שימוש בסיסי
df.groupBy("state").sum("salary").show()

# הצעה 2: עם alias וסינון
dfGrouped = df.groupBy("department") \
    .agg(sum("bonus").alias("sum_bonus"))

dfFilter = dfGrouped.filter(col("sum_bonus") > 100000)
dfFilter.show()

# מיון בסדר יורד
dfFilter.sort(desc("sum_bonus")).show()

# גישת SQL
df.createOrReplaceTempView("EMP")
spark.sql(
    "SELECT state, sum(salary) as sum_salary FROM EMP " +
    "GROUP BY state " +
    "HAVING sum(salary) > 100000 " +
    "ORDER BY sum_salary DESC"
).show()

---

## 🎓 סיכום

בדוגמת PySpark זו הסברתי כיצד לעשות groupBy(), filter() ו-sort() בסדר יורד. אני מקווה שאהבתם את זה.

**לסיכום המושגים שלמדנו:**

✅ **groupBy()** - מקבץ נתונים לפי עמודה או מספר עמודות

✅ **agg()** - מאפשר שימוש בפונקציות אגרגציה מורכבות

✅ **alias()** - נותן שם ברור לעמודות מחושבות

✅ **filter()** - מסנן את התוצאות לפי תנאי

✅ **sort()/orderBy()** - ממיין את התוצאות

✅ **desc()** - מאפשר מיון בסדר יורד

### 📚 מאמרים קשורים
- [PySpark Groupby Count Distinct](https://sparkbyexamples.com/pyspark/pyspark-groupby-count-distinct/)
- [PySpark Groupby on Multiple Columns](https://sparkbyexamples.com/pyspark/pyspark-groupby-on-multiple-columns/)
- [PySpark Groupby Agg (aggregate) – Explained](https://sparkbyexamples.com/pyspark/pyspark-groupby-agg-aggregate/)
- [PySpark GroupBy Count – Explained](https://sparkbyexamples.com/pyspark/pyspark-groupby-count/)
- [PySpark GroupBy Explained with Example](https://sparkbyexamples.com/pyspark/pyspark-groupby-explained-with-example/)
- [PySpark Column alias after groupBy() Example](https://sparkbyexamples.com/pyspark/pyspark-column-alias-after-groupby-example/)
- [PySpark distinct vs dropDuplicates](https://sparkbyexamples.com/pyspark/pyspark-distinct-vs-dropduplicates/)
- [PySpark Get Number of Rows and Columns](https://sparkbyexamples.com/pyspark/pyspark-get-number-of-rows-and-columns/)

---

**Happy Learning! 🚀**